<a href="https://colab.research.google.com/github/siul07/Beugung/blob/main/Beugung.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 14.6 MB/s eta 0:00:00


In [2]:
from google.colab import output
output.enable_custom_widget_manager()

In [3]:
import numpy as np
import matplotlib.pyplot as plt

from ipywidgets import interact, FloatSlider, RadioButtons, VBox, Layout
from IPython.display import display, Math, Markdown

# --- Physik-Funktionen ---

def single_slit_intensity(x, wavelength, slit_width, screen_distance, I0=1.0):
    """
    Intensität I(x) für Beugung am Einzelspalt (Fraunhofer-Näherung).
    """
    theta = x / screen_distance
    beta = np.pi * slit_width * np.sin(theta) / wavelength

    intensity = np.zeros_like(x)
    nonzero = beta != 0
    zero = beta == 0

    intensity[nonzero] = I0 * (np.sin(beta[nonzero]) / beta[nonzero])**2
    intensity[zero] = I0

    return intensity

def double_slit_intensity(x, wavelength, slit_width, slit_distance, screen_distance, I0=1.0):
    """
    Intensität I(x) für Doppelspalt (Interferenz + Einzelspalt-Hüllkurve).
    """
    theta = x / screen_distance
    beta = np.pi * slit_width * np.sin(theta) / wavelength
    delta = np.pi * slit_distance * np.sin(theta) / wavelength

    envelope = np.zeros_like(x)
    nonzero = beta != 0
    zero = beta == 0

    envelope[nonzero] = (np.sin(beta[nonzero]) / beta[nonzero])**2
    envelope[zero] = 1.0

    interference = np.cos(delta)**2
    intensity = I0 * envelope * interference
    return intensity

def make_2d_pattern(intensity_1d, height_pixels=250):
    """
    Erzeuge ein 2D-Beugungsmuster aus einer 1D-Intensitätsverteilung.
    """
    I_norm = intensity_1d / np.max(intensity_1d)
    pattern_2d = np.tile(I_norm, (height_pixels, 1))
    return pattern_2d

# --- Interaktive Anzeige-Funktion ---

def show_diffraction(
    setup='Einzelspalt',
    wavelength_nm=600.0,
    slit_width_um=50.0,
    slit_distance_um=150.0,
    screen_distance_m=1.0,
    x_range_mm=10.0
):
    """
    Visualisiert Intensität + Beugungsmuster und zeigt die Formeln unter dem Plot an.
    """
    # Einheiten umrechnen
    wavelength = wavelength_nm * 1e-9
    slit_width = slit_width_um * 1e-6
    slit_distance = slit_distance_um * 1e-6
    screen_distance = screen_distance_m

    x_max = x_range_mm * 1e-3
    x = np.linspace(-x_max, x_max, 4000)

    if setup == 'Einzelspalt':
        I = single_slit_intensity(x, wavelength, slit_width, screen_distance)
    else:
        I = double_slit_intensity(x, wavelength, slit_width, slit_distance, screen_distance)

    pattern = make_2d_pattern(I, height_pixels=250)

    # --- Plot-Bereich ---
    fig, axes = plt.subplots(
        2, 1, figsize=(8, 6),
        gridspec_kw={'height_ratios': [1, 1.2]}
    )

    # 1D-Plot
    ax1 = axes[0]
    ax1.plot(x * 1e3, I, color='blue')
    ax1.set_xlabel(r'x auf dem Schirm [mm]')
    ax1.set_ylabel(r'I(x) (relativ)')
    ax1.grid(True)
    if setup == 'Einzelspalt':
        ax1.set_title('Einzelspalt – Intensität')
    else:
        ax1.set_title('Doppelspalt – Intensität')

    # 2D-Muster
    ax2 = axes[1]
    extent = [x[0]*1e3, x[-1]*1e3, 0, 1]
    ax2.imshow(pattern, cmap='inferno', aspect='auto', extent=extent, origin='lower')
    if setup == 'Einzelspalt':
        ax2.set_title('Einzelspalt – Beugungsmuster')
    else:
        ax2.set_title('Doppelspalt – Beugungsmuster')
    ax2.set_xlabel('x auf dem Schirm [mm]')
    ax2.set_yticks([])

    plt.tight_layout()
    plt.show()

    # --- Parameter & Formeln anzeigen ---
    display(Markdown("### Eingestellte Parameter"))
    display(Markdown(
        fr"""
- Setup: **{setup}**
- Wellenlänge: \(\lambda = {wavelength_nm:.1f}\,\text{{nm}}\)
- Spaltbreite: \(a = {slit_width_um:.1f}\,\mu\text{{m}}\)
- Spaltabstand: \(d = {slit_distance_um:.1f}\,\mu\text{{m}}\) (nur Doppelspalt)
- Schirmabstand: \(L = {screen_distance_m:.2f}\,\text{{m}}\)
- Schirmbereich: \([-{x_range_mm:.1f}, {x_range_mm:.1f}]\,\text{{mm}}\)
        """
    ))

    if setup == 'Einzelspalt':
        display(Markdown("### Theorie – Einzelspalt"))
        display(Math(
            r"I(x) = I_0 \left( \frac{\sin \beta}{\beta} \right)^2,"
            r"\quad \beta = \frac{\pi a x}{\lambda L}"
        ))
    else:
        display(Markdown("### Theorie – Doppelspalt"))
        display(Math(
            r"I(x) = I_0 \cos^2\left(\frac{\pi d x}{\lambda L}\right)"
            r"\left( \frac{\sin \beta}{\beta} \right)^2,"
            r"\quad \beta = \frac{\pi a x}{\lambda L}"
        ))

# --- Widgets definieren und starten ---

setup_widget = RadioButtons(
    options=['Einzelspalt', 'Doppelspalt'],
    description='Aufbau:',
    layout=Layout(width='200px')
)

wavelength_widget = FloatSlider(
    value=600.0, min=400.0, max=700.0, step=10.0,
    description=r'λ [nm]', continuous_update=False
)
slit_width_widget = FloatSlider(
    value=50.0, min=10.0, max=200.0, step=5.0,
    description='a [µm]', continuous_update=False
)
slit_distance_widget = FloatSlider(
    value=150.0, min=50.0, max=400.0, step=10.0,
    description='d [µm]', continuous_update=False
)
screen_distance_widget = FloatSlider(
    value=1.0, min=0.2, max=3.0, step=0.1,
    description='L [m]', continuous_update=False
)
x_range_widget = FloatSlider(
    value=10.0, min=2.0, max=50.0, step=1.0,
    description='x_max [mm]', continuous_update=False
)

_ = interact(
    show_diffraction,
    setup=setup_widget,
    wavelength_nm=wavelength_widget,
    slit_width_um=slit_width_widget,
    slit_distance_um=slit_distance_widget,
    screen_distance_m=screen_distance_widget,
    x_range_mm=x_range_widget
)

interactive(children=(RadioButtons(description='Aufbau:', layout=Layout(width='200px'), options=('Einzelspalt'…